# Run this cell first

In [ ]:
# this code enables the automated feedback. If you remove this, you won't get any feedback
# so don't delete this cell!
try:
  import AutoFeedback
except (ModuleNotFoundError, ImportError):
  %pip install AutoFeedback
  import AutoFeedback

try:
  from testsrc import test_main
except (ModuleNotFoundError, ImportError):
  %pip install "git+https://github.com/autofeedback-exercises/exercises.git#subdirectory=New-SOR3012/Inhomogeneity"
  from testsrc import test_main

def runtest(tlist):
  import unittest
  from contextlib import redirect_stderr
  from os import devnull
  with redirect_stderr(open(devnull, 'w')):
    suite = unittest.TestSuite()
    for tname in tlist:
      suite.addTest(eval(f"test_main.UnitTests.{tname}"))
    runner = unittest.TextTestRunner()
    try:
      runner.run(suite)
    except AssertionError:
      pass

# Introduction

Markovianity is a pretty extreme assumption to make about a stochastic (random) process. For the reasons discussed in the following video it is usually not realistic.  In this workbook we are thus going to set about relaxing this assumption so you can create more realistic models for your FCT project. 

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/q2843QDkb1Q?si=5_5zwdgieA8KoSko" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

# Revising the poisson process 

Lets start with a litte revision of the ideas that we introduced in the last block of exercises.  In the cell below, I have started writing a class for simulating the Poisson process.  I have thus used the structures you started to use at the end of that workbook to complete a task that you will have done at the start.  The idea is that I should be able to advance time the time stored in the property `self.clock` to the next event by calling the `advance_time` function from the `Poisson` class.  Thre is only one type of event here - arrivals - and the times between these arrivals are generated using the `generate_interarrival` method.

You should be able to use the ideas that were covered in the block on Markov chains in continuous time to complete this code.

Also note that after your class definition that uses your code to produce a graph showing the number of events as a function of time.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Poisson : 
    def __init__(self, lamd) : 
        self.lamd = lamd
        self.clock = 0
        
    def advance_time(self) :
        pass
        
    def generate_interarrival(self) :
        pass
    
# This is my code for generating the graph -- you don't need to modify this
lamd, nevents = 2.0, 100
times, number, g = np.zeros(nevents), np.linspace(1,nevents,nevents), Poisson(lamd)
for i in range(nevents) : 
    g.advance_time()
    times[i] = g.clock
    
plt.plot( times, number, 'ko' )
plt.xlabel("Time / s")
plt.ylabel("Number of events")

# This code is required for the autofeedback- don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_plot'])

# Modifying the poisson process

For a Poisson process the times between events are samples from an exponential distribution.  We use this distribution because it ensures that the Poisson process has the Markov property (i.e. it has no memory).  This distribution is used for service and interarrival times in the M/M/1 queue for the same reason - using the exponential distribution ensures that the queue model has the Markov property.

It is hopefully obvious, however, that it is easy to replace the exponential random variable in the code that we have just written with another type of continuous random variable.  Furhtermore, replacing the exponential random variables with another type of random variable is the first method that we have for generating a stochatic process that does not have the Markov property.

I would like you to use this idea in the following exercise to generate a modified version of the Poisson process code.  Now the interarrival times between events should be samples from the following distribution:

$$
f(x) = \frac{1}{\sqrt{2\pi}} \exp\left( - \frac{(x-4)^2}{2} \right)
$$

I have written code after your class definition that uses your code to produce a graph showing the number of events as a function of time.

In [ ]:
class GeneralPoisson : 
    def __init__(self) : 
        self.clock = 0
        
    def advance_time(self) :
        pass
        
    def generate_interarrival(self) :
        pass
    

# This is my code for generating the graph -- you don't need to modify this
nevents = 100
times, number, g = np.zeros(nevents), np.linspace(1,nevents,nevents), GeneralPoisson()
for i in range(nevents) : 
    g.advance_time()
    times[i] = g.clock
    
plt.plot( times, number, 'ko' )
plt.xlabel("Time / s")
plt.ylabel("Number of events")

# This code is required for the autofeedback- don't delete it!
fighand = plt.gca()


In [ ]:
runtest(['test_plot_2'])

# Setting up rate functions

The video at the start of this workbook introduced the inhomogenous Poisson Process. It is important to note that the code that you have just written with the modified interarrival time **does not** generate an inhomogenous Poisson process.  The stochastic process that this code generated still has the property of homogeneity as the distribution for the interarrival time did not change with time.  There is thus no way that you could use this function to describe a situation such as that in the following statement.

_Customers do not arrive at a uniform rate during the day. There is usually a rush from 9:00-10:00, at lunch from 12:00-13:00, at around 15:00-16:00 and from 18:00-19:00 when folks leave work and head home. During these busy periods, the average rate of arrivals is approximately 30 customers per hour. In less busy times, the rate of arrivals is ten customers per hour._

To deal with this situation we need to allow our random variable parameters to change as a function of time.  In other words, and as discussed in the video, we need to introduce a function $\lambda(t)$ that tells us the rate at difference times of the day.

Given the description above we might use the function:

$$
\lambda(t') = \begin{cases} 
0.5 & \textrm{if} \qquad 0 \le t' < 60 \\
1/6 & \textrm{if} \qquad 60 \le t' < 180 \\
0.5 & \textrm{if} \qquad 180 \le t' < 240 \\
1/6 & \textrm{if} \qquad 240 \le t' < 360 \\
0.5 & \textrm{if} \qquad 360 \le t' < 420 \\
1/6 & \textrm{if} \qquad 420 \le t' < 540 \\
0.5 & \textrm{if} \qquad 5400 \le t' < 600 \\
\end{cases}
$$

for $\lambda(t)$.  Furthermore, to ensure that $t'$ is between 0 and 600 (and thus telling you the time of day) you can do the following transformation

$$
t' = t - 600 \times \textrm{floor}\left( \frac{t}{600} \right)
$$

This ensures that you subtract 600 minutes from the time if the time you are given is at some point on day 2.

Your task in the following exercise is to implement a function called `rate` that takes the time `t` in input and that returns the rate as calculated using the formulas above.

In [ ]:
def rate(t) :
    pass

In [ ]:
runtest(['test_rate_func'])

# Setting up rate functions II

You can include the information on the way the rate of customer arrivals changes throughout the day by using the piecewise function that was employed in the last exercise.  This function is indicated with a black dashed line on the following figure.

![rate-graph.png](https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-SOR3012/Inhomogeneity/rate-graph.png)

You can also use the function that is indicated with a red solid line in this figure.  When this function is used the rate of arrivals peaks in the middle of the busy hours and then decays and builds during the slow times.  Furthermore, the rate of arrivals changes continuously with time.

__Your task in this exercise is to write a function called `rate` in the cell below that can be used determine the rate at each given time.__  Notice that the time that is passed as an argument to the `rate` function is in minutes and not hours as I have used in the axes of the figure above. Notice furthermore that the blue dashed line indicates that the first maximum in the function is at 9:30 in the morning.  Lastly, notice that the rates you return from the `rate` function must be in units of inverse minutes and not units of inverse hours.  

In [ ]:
runtest(['test_rate_func_2'])

# Generating an inhomogenous Poisson process

Now that we understand how to write functions with time dependent rates we are in a position to learn how to write code to generate the Inhomogeneous Poisson process that was discussed in the video.  In the following example we are going to suppose that our rate function is:

$$
\lambda(t) = \sin(t) + 2
$$

The maximum rate is thus 3, while the minimum is 1. Notice that the algorithm that I am going to introduce here requires you to use a rate function that is bounded from above and below.  Furthermore, the rate function should be positive for all values of $t$.

The algorithm works as follows:

1. You generate phantom events using a Poisson process with the maximum rate.  
2. Now suppose that the a phantom event occurs at time $t$. You then generate a uniform random variable between 0 and 1.  You accept that the new event really happens at time $t$ with a probability that is equal $\lambda(t)/\lambda_\textrm{max}$.  In other words, you generate a uniform random variable between 0 and 1, $U$ and accept the event if $U\le \lambda(t)/\lambda_\textrm{max}$ 

If I were writing Python code to generate interarrival times in this way, I would use a Python code something like the following:

```python
while True : 
    # This line generates my phantom events
    t = t + exponential(lmax)
    # This tells whether or not this is the real event.  
    # If it is I break out of the while loop and stop incrementing the time.
    if np.random.uniform(0,1) <= rate(t)/lmax : break
```

With all that in mind, your task below is to modify the code below in order to implement the inhomgeneous Poisson process that I have described above.  As before I have written some code to generate a graph showing the number of events as a function of time for you.

In [ ]:
class InhomogenousPoisson : 
    def __init__(self) : 
        self.clock = 0
        
    def rate(self,t) :
        return np.sin(t) + 2
        
    def advance_time(self) :
        pass
        
    def generate_interarrival(self) :
        pass
    
# This is my code for generating the graph -- you don't need to modify this
nevents = 100
times, number, g = np.zeros(nevents), np.linspace(1,nevents,nevents), InhomogenousPoisson()
for i in range(nevents) : 
    g.advance_time()
    times[i] = g.clock
    
plt.plot( times, number, 'ko' )
plt.xlabel("Time / s")
plt.ylabel("Number of events")

In [ ]:
runtest(['test_plot_3'])

# Taking it further

Much like in the last block, I am looking for you to try and use these ideas for some aspect of your FCT project.  Notice that for queue's although we have focussed on making the interarrival times inhomgeneous in the exercises here you can also make the service times inhomogeneous.  You can even use ideas from this notebook to make the service times non-random!  In short, you now have a lot of modelling ideas that you can play around with so please get creative!   